In [1]:
import torch
import math
torch.set_grad_enabled(False)

# LINEAR

In [123]:
#LINEAR MODULE (FULLY CONNECTED LAYER)
#mini batch : x[mini_batch_size * D]D=2
class Linear(object):
    def __init__(self, dimension, nb_data_in, nb_data_out):
        #x = nb_data_in x dim in my calculation, but here transpose (dim x nb_data_in)
        #y = nb_data_out x dim in my calculation, but here transpose (dim x nb_data_out)
        k = math.sqrt(1/nb_data_in)
        self.weight = torch.empty(nb_data_out,nb_data_in).uniform_(-k,k)
        self.bias = torch.empty(nb_data_out,dimension).uniform_(-k,k)
        self.grad_weight = None
        self.grad_bias = None
        self.input = None
        
    def updateparam(self, lr):
        for i in range(len(g_w)): 
            self.weight -= lr * self.grad_weight[i]
            self.bias -= lr * self.grad_bias[i]
    
    def forward(self , *input, nb_layer):
        #All the calculation were done considering x = nb_data_in x dim (as seen in the lesson)
        #So to simplify the comprehension, we do a first transpose, do the the calculations with this
        #then transpose again after, to have the correct dimension output

        #save x with dim = dim x nb_data_in
        if(nb_layer > 1):
            input = input[0]
            #pour le premier, on aura en entrée un ou plusieurs tensors (donne tuple quand rentré dans f)
        
        #save x with dim = dim x nb_data_in    
        self.input = input
        output = []
        
        for x in input:
            #x = dim x nb_data_in -> nb_data_in x dim
            x_correct_dim = x.t()
            #y = nb_data_out x dim
            y_correct_dim = None

            #dim analysis: [nb_data_out x nb_data_in] x [nb_data_in x dim] + [nb_data_out x dim]
            y_correct_dim = ((self.weight).matmul(x_correct_dim)+self.bias)

            #append y = [dim x nb_data_out] to respect lin module
            output.append(y_correct_dim.t()) 
        return tuple(output)
    
    def backward(self, *gradwrtoutput):
        gradwrtoutput = gradwrtoutput[0]
        dl_dx  = None
        dl_dw = []
        dl_db = []
        gradaccumulated = []
        
        for i in range (len(gradwrtoutput)):
            #X = dim x nb_data_in -> nb_data_in x dim
            x_correct_dim = self.input[i].t()
            
            #X = dim x nb_data_out -> nb_data_out x dim
            grad_correct_dim = gradwrtoutput[i].t()
            
            #dl_dx = w^T x dl_dy (gradwrtoutput)
            #dim analysis: nb_data_in x dim = [nb_data_in x nb_data_out] x [nb_data_out x dim]
            dl_dx = (self.weight).t().matmul(grad_correct_dim)
            #nb_data_in x dim -> dim x nb_data_in
            gradaccumulated.append(dl_dx.t())
            
            #dl_db = dl_dy
            #dim analysis: nb_data_out x dim (car b = nb_data_out x dim)
            dl_db.append(grad_correct_dim)
            
            #dl_dw = dl_dy x X^T
            #dim analysis: nb_data_out x nb_data_in = [nb_data_out x dim] x [dim x nb_data_in]
            dl_dw.append(grad_correct_dim.matmul(x_correct_dim.t()))
            
        self.grad_weight = dl_dw
        self.grad_bias = dl_db

        return tuple(gradaccumulated)
        
    def param(self):
        output = [[self.weight, self.grad_weight], [self.bias, self.grad_bias]]
        return output
    
    #https://pytorch.org/docs/stable/nn.html#linear-layers

In [81]:
a = torch.empty(1,2).normal_()
l = []
l.append(a)
i = tuple(l)
print('input')
print(i)

print('dim = ', i[0].size(0),' input =', i[0].size(1))
nb_out = 3
print('nb_out = ', nb_out)

lin = Linear(i[0].size(0),i[0].size(1),nb_out)
f = lin.forward(i,nb_layer = 2)
print('forward')
print(f)

input
(tensor([[0.4168, 0.6865]]),)
dim =  1  input = 2
nb_out =  3
forward
(tensor([[ 0.2704, -0.6901, -0.2762]]),)


In [82]:
g = []
grad = torch.empty(i[0].size(0),nb_out).normal_()
g.append(grad)
gr = tuple(g)
print('grad')
print(gr)
output = lin.backward(gr)
print('gradaccumulated')
print(output)

grad
(tensor([[ 0.2142,  1.3402, -0.6543]]),)
gradaccumulated
(tensor([[-0.0925, -0.2774]]),)


# RELU

In [83]:
#RELU MODULE
class ReLU():
    def __init__(self):
        self.input = None
    
    def forward(self, *input):
        input = input[0]
        liste = []
        self.input = input

        for x in input:
            liste.append(torch.max(x,torch.zeros_like(x))) 
        output = tuple(liste)
        return output
        
    def backward(self, *gradwrtoutput): 
        #add input as param ? 
        #No, parameters are w and b when linear, but input is saved has if we have to give backward and input
        #in the case they're both tuple of tensors of size not know
        #it's complicated (maybe impossible) to code it with the * (let the size be whatever we want)
        gradwrtoutput = gradwrtoutput[0]
        derivative = []
        gradaccumulated = []

        for x in self.input:
            dx = (x>=0).float()
            derivative.append(dx)
        for i in range (len(derivative)):
            gradaccumulated.append(derivative[i]*gradwrtoutput[i])
        output = tuple(gradaccumulated)
        return output   

    def param(self): 
        return []
    
#backward should get as input a tensor or a tuple of tensors containing the gradient of the 
#loss with respect to the module’s output, accumulate the gradient wrt the parameters, 
#and return a tensor or a tuple of tensors containing the gradient of the loss wrt the module’s input.

In [84]:
a = torch.empty(1,2).normal_()
l = []
l.append(a)
i = tuple(l)
print('input')
print(i)

print('dim = ', i[0].size(0),' input =', i[0].size(1))
nb_out = 3
print('nb_out = ', nb_out)

lin = Linear(i[0].size(0),i[0].size(1),nb_out)
f = lin.forward(i,nb_layer = 2)
print('forward linear')
print(f)

R = ReLU()
r = R.forward(f)
print('forward ReLU')
print(r)

input
(tensor([[ 0.6993, -0.6388]]),)
dim =  1  input = 2
nb_out =  3
forward linear
(tensor([[ 0.3345,  0.3300, -0.4223]]),)
forward ReLU
(tensor([[0.3345, 0.3300, 0.0000]]),)


In [85]:
g = []
grad = torch.empty(r[0].size(0),r[0].size(1)).normal_()
g.append(grad)
gr = tuple(g)
print('grad')
print(gr)

output = R.backward(gr)
print('grad acc ReLu')
print(output)

out = lin.backward(output)
print('grad acc Lin')
print(out)

grad
(tensor([[ 0.4437,  0.4750, -0.5213]]),)
grad acc ReLu
(tensor([[0.4437, 0.4750, -0.0000]]),)
grad acc Lin
(tensor([[ 0.4498, -0.1868]]),)


# TANH

In [138]:
#TANH MODULE
class Tanh():
    def __init__(self):
        self.input = None
        
    def forward(self, *input):
        input = input[0]
        output = []
        self.input = input
        for x in input:
            output.append(torch.tanh(x)) 
        return tuple(output)
    
    def backward(self, *gradwrtoutput):
        gradwrtoutput = gradwrtoutput[0]
        output = []
        for i in range (len(gradwrtoutput)):
            # derivative = (1 - torch.tanh(x).pow(2))
            output.append((1 - torch.tanh(self.input[i]).pow(2))*gradwrtoutput[i])
        return tuple(output)   

    def param(self):
        return [] #Pas de param

In [87]:
a = torch.empty(1,2).normal_()
l = []
l.append(a)
i = tuple(l)
print('input')
print(i)

print('dim = ', i[0].size(0),' input =', i[0].size(1))
nb_out = 3
print('nb_out = ', nb_out)

lin = Linear(i[0].size(0),i[0].size(1),nb_out)
f = lin.forward(i,nb_layer = 2)
print('forward linear')
print(f)

T = Tanh()
t = T.forward(f)
print('forward Tanh')
print(t)

input
(tensor([[ 0.7674, -1.0612]]),)
dim =  1  input = 2
nb_out =  3
forward linear
(tensor([[-0.9336, -0.2106,  0.9268]]),)
forward Tanh
(tensor([[-0.7323, -0.2075,  0.7291]]),)


In [88]:
g = []
grad = torch.empty(r[0].size(0),r[0].size(1)).normal_()
g.append(grad)
gr = tuple(g)
print('grad')
print(gr)

output = T.backward(gr)
print('grad acc Tanh')
print(output)

out = lin.backward(output)
print('grad acc Lin')
print(out)

grad
(tensor([[-0.4403, -0.3640, -1.7230]]),)
grad acc Tanh
(tensor([[-0.2042, -0.3483, -0.8071]]),)
grad acc Lin
(tensor([[-0.5901, -0.0546]]),)


# LOSSMSE

In [89]:
#LOSSMSE MODULE
    
class LossMSE():
    #def __init__(self):
        #Besoin d'aucun je pense
        #self.loss = None
        #self.gradloss = None
    
    def forward(self, *input, target): 
        input = input[0]
        #Si plusieurs input et target comment on fait?
        #On aura un tuple de tensor qui consistera l'input, puis target = tuple of tensor target
        liste = torch.Tensor([])
        #Utilisation de cette écriture pour pouvoir faire somme sur "liste" (l = [] marche pas avec sum())
        for i in range (len(input)):
            liste = torch.cat((liste,(input[i]-target[i]).pow(2)))
        loss = torch.sum(liste)/len(input)
        #self.loss = loss même pas besoin
        return loss
        
    def backward(self, *input, target):
        gradaccumulated = []
        input = input[0]
        dloss = []
        
        for i,x in enumerate(input):
            dloss.append(2*(x - target[i])/len(input))
        return tuple(dloss)

    def param(self):
        return [] #No param

In [90]:
N = LossMSE()
x = torch.empty(1,2).fill_(1)
y = torch.empty(1,2).fill_(2)
l = []
l.append(x)
l.append(y)
i = tuple(l)
print("input")
print(i)

t1 = torch.empty(1,2).fill_(-1)
t2 = torch.empty(1,2).fill_(-1)
ta = []
ta.append(t1)
ta.append(t2)
t = tuple(ta)
print("target")
print(t)
lo = N.forward(i, target = t)
print('loss')
print(lo)
dl = N.backward(i, target= t)
print('dloss')
print(dl)

input
(tensor([[1., 1.]]), tensor([[2., 2.]]))
target
(tensor([[-1., -1.]]), tensor([[-1., -1.]]))
loss
tensor(13.)
dloss
(tensor([[2., 2.]]), tensor([[3., 3.]]))


# Test 1 layer

In [91]:
a = torch.empty(1,2).normal_()
l = []
l.append(a)
i = tuple(l)
print('input')
print(i)

print('dim = ', i[0].size(0),' input =', i[0].size(1))
nb_out = 3
print('nb_out = ', nb_out)

lin = Linear(i[0].size(0),i[0].size(1),nb_out)
f = lin.forward(i,nb_layer = 2)
print('forward linear')
print(f)

T = Tanh()
output = T.forward(f)
print('forward Tanh')
print(output)

input
(tensor([[-0.1575, -1.2043]]),)
dim =  1  input = 2
nb_out =  3
forward linear
(tensor([[-0.0053, -0.3449,  0.0054]]),)
forward Tanh
(tensor([[-0.0053, -0.3319,  0.0054]]),)


In [92]:
N = LossMSE()
t1 = torch.empty(output[0].size(0),output[0].size(1)).fill_(-1)

ta = []
ta.append(t1)
t = tuple(ta)
print("target")
print(t)

lo = N.forward(output, target = t)
print('loss')
print(lo)
dl = N.backward(output, target= t)
print('dloss')
print(dl)

backt = T.backward(dl)
print('grad acc Tanh')
print(backt)

gradtot = lin.backward(backt)
print('grad acc Lin')
print(gradtot)

target
(tensor([[-1., -1., -1.]]),)
loss
tensor(2.4467)
dloss
(tensor([[1.9895, 1.3362, 2.0108]]),)
grad acc Tanh
(tensor([[1.9894, 1.1891, 2.0108]]),)
grad acc Lin
(tensor([[1.7043, 1.6799]]),)


In [93]:
[w,g_w],[b,g_b] = lin.param()
print(w)
print(g_w)
print(b)
print(g_b)

tensor([[0.1713, 0.3169],
        [0.5001, 0.4302],
        [0.3824, 0.2675]])
[tensor([[-0.3134, -2.3958],
        [-0.1873, -1.4319],
        [-0.3168, -2.4215]])]
tensor([[0.4034],
        [0.2519],
        [0.3878]])
[tensor([[1.9894],
        [1.1891],
        [2.0108]])]


# Test 1 layer SGD

In [107]:
a = torch.empty(1,2).normal_()
l = []
l.append(a)
i = tuple(l)
print('input')
print(i)

print('dim = ', i[0].size(0),' input =', i[0].size(1))
nb_out = 3
print('nb_out = ', nb_out)

lin = Linear(i[0].size(0),i[0].size(1),nb_out)
f = lin.forward(i,nb_layer = 2)
print('forward linear')
print(f)

T = Tanh()
output = T.forward(f)
print('forward Tanh')
print(output)

N = LossMSE()
t1 = torch.empty(output[0].size(0),output[0].size(1)).fill_(-1)

print()
ta = []
ta.append(t1)
t = tuple(ta)
print("target")
print(t)

lo = N.forward(output, target = t)
print('loss')
print(lo)
dl = N.backward(output, target= t)
print('dloss')
print(dl)

backt = T.backward(dl)
print('grad acc Tanh')
print(backt)

gradtot = lin.backward(backt)
print('grad acc Lin')
print(gradtot)

print()
print('learning rate')
lr = 0.1
print(lr)
[[wp,g_w],[bp,g_b]] = lin.param()
print('w, g_w, b and g_b')
print(wp)
print(g_w)
print(bp)
print(g_b)

print()

lin.updateparam(lr)

for i in range(len(g_w)): 
    wp += lr * g_w[i]
    bp += lr * g_b[i]
    
print('w and b updated manually')
print(wp)
print(bp)

print('w and b updated function')
[[w,g_w],[b,g_b]] = lin.param()
print(w)
print(b)

input
(tensor([[-2.2146, -0.6439]]),)
dim =  1  input = 2
nb_out =  3
forward linear
(tensor([[1.8012, 1.6285, 1.9005]]),)
forward Tanh
(tensor([[0.9469, 0.9258, 0.9563]]),)

target
(tensor([[-1., -1., -1.]]),)
loss
tensor(11.3265)
dloss
(tensor([[3.8939, 3.8517, 3.9126]]),)
grad acc Tanh
(tensor([[0.4023, 0.5500, 0.3347]]),)
grad acc Lin
(tensor([[-0.6299, -0.3469]]),)

learning rate
0.1
w, g_w, b and g_b
tensor([[-0.5408,  0.1284],
        [-0.4650, -0.3815],
        [-0.4679, -0.5637]])
[tensor([[-0.8909, -0.2590],
        [-1.2181, -0.3542],
        [-0.7412, -0.2155]])]
tensor([[0.6862],
        [0.3531],
        [0.5013]])
[tensor([[0.4023],
        [0.5500],
        [0.3347]])]

tensor([[-0.5408,  0.1284],
        [-0.4650, -0.3815],
        [-0.4679, -0.5637]])
tensor([[0.6862],
        [0.3531],
        [0.5013]])
w and b updated manually
tensor([[-0.7190,  0.0766],
        [-0.7086, -0.4524],
        [-0.6161, -0.6068]])
tensor([[0.7667],
        [0.4631],
        [0.5682]])


# Gradient 2 layer Tanh

In [156]:
a = torch.empty(1,2).normal_()
l = []
l.append(a)
inp = tuple(l)
print('input')
print(inp)

print('dim = ', inp[0].size(0),' input =', inp[0].size(1))
nb_out1 = 3
print('to dim = ', inp[0].size(0),' nb_out_1 =', nb_out1)
nb_out2 = 4
print('to dim = ', inp[0].size(0),' nb_out_2 =', nb_out2)


nb_iterations = 25
print(nb_iterations)

lin1 = Linear(inp[0].size(0),inp[0].size(1),nb_out1)
lin2 = Linear(inp[0].size(0),nb_out1,nb_out2)
T1 = Tanh()
T2 = Tanh()
N = LossMSE()

for i in range (nb_iterations):
    print()
    print('iteration', i)
    
    f1 = lin1.forward(inp,nb_layer = 2)
    t1 = T1.forward(f1)
    f2 = lin2.forward(t1,nb_layer = 3)
    output = T2.forward(f2)

    t1 = torch.empty(output[0].size(0),output[0].size(1)).fill_(-1)
    ta = []
    ta.append(t1)
    t = tuple(ta)
    lo = N.forward(output, target = t)
    print('loss')
    print(lo)
    dl = N.backward(output, target= t)
    
    bt2 = T2.backward(dl)
    bf2 = lin2.backward(bt2)
    bt1 = T1.backward(bf2)
    bf1 = lin1.backward(bt1)

    lr = 0.1
    lin1.updateparam(lr)
    lin2.updateparam(lr)

input
(tensor([[-0.8601,  2.3773]]),)
dim =  1  input = 2
to dim =  1  nb_out_1 = 3
to dim =  1  nb_out_2 = 4
25

iteration 0
loss
tensor(4.7981)

iteration 1
loss
tensor(3.2041)

iteration 2
loss
tensor(1.7052)

iteration 3
loss
tensor(0.3651)

iteration 4
loss
tensor(0.2186)

iteration 5
loss
tensor(0.1624)

iteration 6
loss
tensor(0.1304)

iteration 7
loss
tensor(0.1093)

iteration 8
loss
tensor(0.0941)

iteration 9
loss
tensor(0.0828)

iteration 10
loss
tensor(0.0738)

iteration 11
loss
tensor(0.0667)

iteration 12
loss
tensor(0.0608)

iteration 13
loss
tensor(0.0558)

iteration 14
loss
tensor(0.0516)

iteration 15
loss
tensor(0.0480)

iteration 16
loss
tensor(0.0449)

iteration 17
loss
tensor(0.0421)

iteration 18
loss
tensor(0.0397)

iteration 19
loss
tensor(0.0375)

iteration 20
loss
tensor(0.0355)

iteration 21
loss
tensor(0.0338)

iteration 22
loss
tensor(0.0322)

iteration 23
loss
tensor(0.0307)

iteration 24
loss
tensor(0.0294)


In [ ]:
# Gradient 2 layer ReLU

In [160]:
a = torch.empty(1,2).normal_()
l = []
l.append(a)
inp = tuple(l)
print('input')
print(inp)

print('dim = ', inp[0].size(0),' input =', inp[0].size(1))
nb_out1 = 3
print('to dim = ', inp[0].size(0),' nb_out_1 =', nb_out1)
nb_out2 = 4
print('to dim = ', inp[0].size(0),' nb_out_2 =', nb_out2)


nb_iterations = 25
print(nb_iterations)

lin1 = Linear(inp[0].size(0),inp[0].size(1),nb_out1)
lin2 = Linear(inp[0].size(0),nb_out1,nb_out2)
R1 = ReLU()
R2 = ReLU()
N = LossMSE()

for i in range (nb_iterations):
    print()
    print('iteration', i)
    
    f1 = lin1.forward(inp,nb_layer = 2)
    t1 = R1.forward(f1)
    f2 = lin2.forward(t1,nb_layer = 3)
    output = R2.forward(f2)

    t1 = torch.empty(output[0].size(0),output[0].size(1)).fill_(-1)
    ta = []
    ta.append(t1)
    t = tuple(ta)
    lo = N.forward(output, target = t)
    print('loss')
    print(lo)
    dl = N.backward(output, target= t)
    
    br2 = R2.backward(dl)
    bf2 = lin2.backward(br2)
    br1 = R1.backward(bf2)
    bf1 = lin1.backward(br1)

    lr = 0.01
    lin1.updateparam(lr)
    lin2.updateparam(lr)

input
(tensor([[ 0.7204, -1.3981]]),)
dim =  1  input = 2
to dim =  1  nb_out_1 = 3
to dim =  1  nb_out_2 = 4
25

iteration 0
loss
tensor(4.0806)

iteration 1
loss
tensor(4.0346)

iteration 2
loss
tensor(4.)

iteration 3
loss
tensor(4.)

iteration 4
loss
tensor(4.)

iteration 5
loss
tensor(4.)

iteration 6
loss
tensor(4.)

iteration 7
loss
tensor(4.)

iteration 8
loss
tensor(4.)

iteration 9
loss
tensor(4.)

iteration 10
loss
tensor(4.)

iteration 11
loss
tensor(4.)

iteration 12
loss
tensor(4.)

iteration 13
loss
tensor(4.)

iteration 14
loss
tensor(4.)

iteration 15
loss
tensor(4.)

iteration 16
loss
tensor(4.)

iteration 17
loss
tensor(4.)

iteration 18
loss
tensor(4.)

iteration 19
loss
tensor(4.)

iteration 20
loss
tensor(4.)

iteration 21
loss
tensor(4.)

iteration 22
loss
tensor(4.)

iteration 23
loss
tensor(4.)

iteration 24
loss
tensor(4.)


# SEQUENTIAL